In [159]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.path as pth
pd.options.display.max_colwidth=300
pd.options.display.max_columns=30
pd.options.display.max_rows=300

In [10]:
cwd= '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files'
os.chdir(cwd)
#files=glob.glob('*.txtCB')
#files

In [3]:
for f in files:
    file_name, file_ext=os.path.splitext(f)
    file_name2, ext2=os.path.splitext(file_name)
    new_name=(f'{file_name2}' + 'CB.txt')
    os.rename(f, new_name)

In [2]:
# %load '/home/balla-gyula/Dokumentumok/Scripts/chooselines.py'
'''
Takes a text file and  returns the text it contains except the specified number of lines in the beginning.
For eg. with storm roiattr files we can cut off the lines not containing roi coordinates: chooselines
(chooseline('path/to/file/roiattr.txt', 11)
'''
def chooselines(path, n): # takes the path to file and the number of lines to cut off from the beginning
    with open(path, 'r')as f:
        line_offset= []
        offset=0
        for line in f:
            line_offset.append(offset)
            offset+=len(line)+1
        f.seek(0)
        f.seek(line_offset[n])
        fname=os.path.splitext(path)[0]
        newfile=str(fname + '_COORD.txt')
        with open(newfile, 'w') as wf:
            wf.write('x\ty\n')
            aa=f.read()
            wf.write(aa)
        rf=f.read()
        return(print(rf))



### Creating coordinate files for polygons from the GABA roi text files

In [14]:
attrwd='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/'
os.chdir(attrwd)
gaba_attr=glob.glob('*RoiAttr.txt')
for file in gaba_attr:
    chooselines(file, 11)

### Creating coordinate files for polygons from the glut roi text files

In [22]:
attrwd='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_glut_files//'
os.chdir(attrwd)
glut_attr=glob.glob('*RoiAttr.txt')
for file in glut_attr:
    chooselines(file, 11)

In [3]:
def create_RoiCoord(path_poly, path_stormfile):               
    '''
    Takes a 2D polygon and a storm file (with one or multiple channels)
    and returnes a csv with the localization points coordinates inside the polygon.
    '''
    polygon=pd.read_csv(path_poly, sep='\t', header='infer')  # defines polygon
    storm=pd.read_csv(path_stormfile, sep='\t', header='infer',index_col='Channel Name')              
                                                              # draws storm LPs
    channels=['405/561', '405/647', '405/561AL']              # selected channels
    storm=storm.loc[channels]                                 # filters for desired channels

    storm_xyarr=list(zip(storm['Xwc'], storm['Ywc']))         # convert storm x and y coordinates to numpy arrays
    storm_xyarr=np.array(storm_xyarr)

    contain=pth.Path(polygon).contains_points(storm_xyarr)    # gives logical values if the given point is inside the polygon
    storm['IN_poly']=contain
    in_poly=storm[storm['IN_poly']==True]
    
    stormname, stormext=os.path.splitext(path_stormfile)      # split file path+name and extension
    print(stormname)
    filename=(stormname + '_ROI.txt')
    in_poly.to_csv(path_or_buf=filename,
                  sep='\t')                                   # save dataframe


In [4]:
create_RoiCoord(path_poly='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01_activeContourROI_4_RoiAttr_COORD.txt', 
                path_stormfile='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt')

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB


In [24]:
gaba_pivot=pd.read_csv('/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/code/gaba_roi_pivot.csv',
          index_col='gaba_slice')
gaba_pivot['path']='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/'
gaba_pivot

,gaba_roi,path
gaba_slice,,
BB185_02_01,['BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01_activeContourROI_4_RoiAttr.txt'],/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_02_02,"['BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02_activeContourROI_13_RoiAttr.txt', 'BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02_activeContourROI_16_RoiAttr.txt']",/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_02_03,"['BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03_activeContourROI_19_RoiAttr.txt', 'BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03_activeContourROI_24_RoiAttr.txt']",/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_03_01,['BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05_activeContourROI_27_RoiAttr.txt'],/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_03_02,"['BB185_03_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S06_activeContourROI_34_RoiAttr.txt', 'BB185_03_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S06_activeContourROI_37_RoiAttr.txt']",/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_03_03,['BB185_03_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S07_activeContourROI_40_RoiAttr.txt'],/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_03_04,['BB185_03_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S08_activeContourROI_43_RoiAttr.txt'],/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_03_05,['BB185_03_934_CB1_647_1116_Bsn_568_storm05_list-2018-06-25-19-18-35_S09_activeContourROI_46_RoiAttr.txt'],/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/
BB185_04_01,"['BB185_04_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S10_activeContourROI_49_RoiAttr.txt', 'BB185_04_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S10_activeContourROI_54_RoiAttr.txt']",/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/


### Creating a Dataframe listing the names of the combined (quad CB1+Basson and red Basson) files, and indexing it based on the filename. 
### The format of the indexing is the following: BB{xxx}\_{yy}\_{zz}, where:
- xxx - cell number
- yy - slice numbe
- zz - bouton number

In [222]:
filescb= !ls /home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/
cbf=pd.DataFrame(data=filescb, columns=['filenames'])
cbf['slc']=cbf['filenames'].map(lambda x: x[:9] + x[39:41])
cbf.set_index('slc', inplace=True)
cbf.head()

,filenames
slc,
BB185_02_01,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt
BB185_02_02,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt
BB185_02_03,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt
BB185_02_04,BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04CB.txt
BB185_03_01,BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05CB.txt


### Creating a list with the RoiAttr files, indexing it in a similar way as before. Then mapping the corresponding combined storm filenames in a separate column.
### Finally creating a Dataframe without missing data either in because of the RoiAttr or the combined storm files.

In [221]:
os.chdir('../Roiattr_GABA_files/')
filesattr=!ls *COORD.txt                                                # list roi coordinates
attrf=pd.DataFrame(data=filesattr, columns=['attrnames'])
attrf['slice']=attrf['attrnames'].map(lambda x: x[:9] + x[39:41])       # creating index names
attrf.set_index('slice', inplace=True)
attrf['filenames']=attrf.index.map(cbf['filenames'])                    # mapping the corresponding filename to the attr names
attrf_dropna=attrf.dropna(how='any')                                    #removing nans
attrf_dropna.head()

,attrnames,filenames
slice,,
BB185_02_01,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01_activeContourROI_4_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt
BB185_02_02,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02_activeContourROI_13_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt
BB185_02_02,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02_activeContourROI_16_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt
BB185_02_03,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03_activeContourROI_19_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt
BB185_02_03,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03_activeContourROI_24_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt


In [179]:
os.chdir('/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/')
os.makedirs('RoiCoord_GABA_files')

FileExistsError: [Errno 17] File exists: 'RoiCoord_GABA_files'

In [209]:
# file paths
os.chdir('/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/RoiCoord_GABA_files/')
path_attr='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_GABA_files/'
path_files='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/'

In [211]:
# list containing the names of the attr files and the correspondig storm files
attrlist=list(path_attr + attrf_dropna['attrnames'])
filelist=list(path_files + attrf_dropna['filenames'])
filelist

['/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_11

### Creating coordinate text files from the points inside the polygons and saving it as *ROI.txt files

In [217]:
for x, y in zip(attrlist, filelist):
    create_RoiCoord(x, y)

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S53CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S53CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S53CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S54CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S54CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB192_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_05_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S43CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_05_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S43CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_05_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S44CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_05_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S44CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_05_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S44CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-14-09-03_S04CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-14-09-03_S04CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-03_S05CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-03_S05CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-03_S05CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB203_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-14-09-0

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S86CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S86CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S87CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S87CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S88CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-03_S08CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-14-09-03_S09CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-14-09-03_S09CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-14-09-03_S09CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm04_list-2018-07-18-14-09-03_S10CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm04_list-2018-07-18-14-09-0

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-15-30-01_S01CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-15-30-01_S01CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-15-30-01_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-15-30-01_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-15-30-01_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB238_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-15-30-0

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-16-35-11_S05CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-16-35-11_S06CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-16-35-11_S06CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-16-35-11_S06CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-16-35-11_S07CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-16-35-1

## The same with glutamatergic terminals

In [223]:
filescb= !ls /home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/
cbf=pd.DataFrame(data=filescb, columns=['filenames'])
cbf['slc']=cbf['filenames'].map(lambda x: x[:9] + x[39:41])
cbf.set_index('slc', inplace=True)
cbf.head()

,filenames
slc,
BB185_02_01,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt
BB185_02_02,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt
BB185_02_03,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt
BB185_02_04,BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04CB.txt
BB185_03_01,BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05CB.txt


In [242]:
os.chdir('../Roiattr_glut_files/')
filesattr_glu=!ls *COORD.txt                                                # list roi coordinates
attrf_glu=pd.DataFrame(data=filesattr_glu, columns=['attrnames'])
attrf_glu['slice']=attrf_glu['attrnames'].map(lambda x: x[:9] + x[39:41])       # creating index names
attrf_glu.set_index('slice', inplace=True)
attrf_glu['filenames']=attrf_glu.index.map(cbf['filenames'])                    # mapping the corresponding filename to the attr names
attrf_glu_dropna=attrf_glu.dropna(how='any')                                    #removing nans
attrf_glu_dropna

,attrnames,filenames
slice,,
BB185_02_01,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01_activeContourROI_2_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt
BB185_02_02,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02_activeContourROI_7_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt
BB185_02_03,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03_activeContourROI_15_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt
BB185_02_04,BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04_activeContourROI_18_RoiAttr_COORD.txt,BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04CB.txt
BB185_03_01,BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05_activeContourROI_21_RoiAttr_COORD.txt,BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05CB.txt
BB185_03_02,BB185_03_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S06_activeContourROI_24_RoiAttr_COORD.txt,BB185_03_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S06CB.txt
BB185_03_03,BB185_03_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S07_activeContourROI_27_RoiAttr_COORD.txt,BB185_03_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S07CB.txt
BB185_03_04,BB185_03_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S08_activeContourROI_30_RoiAttr_COORD.txt,BB185_03_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S08CB.txt
BB185_03_05,BB185_03_934_CB1_647_1116_Bsn_568_storm05_list-2018-06-25-19-18-35_S09_activeContourROI_33_RoiAttr_COORD.txt,BB185_03_934_CB1_647_1116_Bsn_568_storm05_list-2018-06-25-19-18-35_S09CB.txt


In [243]:
os.chdir('/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/')
os.makedirs('RoiCoord_glut_files')

In [244]:
# file paths for glutamatergic terminals
os.chdir('/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/RoiCoord_glut_files/')
path_attr_glu='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/Roiattr_glut_files/'
path_files_glu='/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/'

In [248]:
# list containing the names of the attr files and the correspondig storm files
attrlist_glu=list(path_attr_glu + attrf_glu_dropna['attrnames'])
filelist=list(path_files + attrf_glu_dropna['filenames'])
filelist

['/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05CB.txt',
 '/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_11

In [249]:
for x, y in zip(attrlist_glu, filelist):
    create_RoiCoord(x, y)

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S01CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S03CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_02_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S04CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S05CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB185_03_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_01_934_CB1_647_1116_Bsn_568_storm02_list-2018-06-25-19-18-35_S34CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_01_934_CB1_647_1116_Bsn_568_storm03_list-2018-06-25-19-18-35_S35CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_01_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S36CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_01_934_CB1_647_1116_Bsn_568_storm04_list-2018-06-25-19-18-35_S36CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_02_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-35_S38CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB193_03_934_CB1_647_1116_Bsn_568_storm01_list-2018-06-25-19-18-3

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_05_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-03_S01CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_05_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-14-09-03_S02CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_05_934_CB1_647_1116_Bsn_568_storm04_list-2018-07-18-14-09-03_S03CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-14-09-03_S07CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-14-09-03_S07CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB226_06_934_CB1_647_1116_Bsn_568_storm02_list-2018-07-18-14-09-0

/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_06_934_CB1_647_1116_Bsn_568_storm03_list-2018-07-18-16-35-11_S07CB
/home/balla-gyula/mnt/Csoport43-2/Gyula/STORM_alignment_investigation_data/data/combined_files/BB239_07_934_CB1_647_1116_Bsn_568_storm01_list-2018-07-18-16-35-11_S08CB
